In [1]:
import pickle
import numpy as np
from gensim.models import Doc2Vec as d2v
from gensim.models.doc2vec import TaggedDocument as td
import pandas as pd
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import pandas as pd

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif, mutual_info_classif

from pprint import pprint
from time import time
import logging

In [2]:
#load all of our documents
with open('NHlegtext09_20.corpus','rb') as f:
    legcorpus = pickle.load(f)
    
# #print a random doc
# print(legcorpus[303])

#assemble the text data, and remove the part where it says if it passes or not
combined_docs = {} #{bill_id:text}
total = 0
passed = 0

#a more unorthodox method of grabbing the target passage
leg_target = {} #{bill: 0 or 1, where 1 means pass}

#for each document, check if it is approved, record that, then combine the list into a string
for doc in legcorpus:
    pass_flag = 0
    if doc[0] not in combined_docs:
        total += 1
        try:
            if doc[2][-5] == 'approved':
                doc[2] = doc[2][:-5]
                passed += 1
                pass_flag = 1
                combined_docs[doc[0]] = " ".join(doc[2])
            else:
                combined_docs[doc[0]] = " ".join(doc[2])
                
        except:
            combined_docs[doc[0]] = " ".join(doc[2])
            pass
                
    leg_target[doc[0]] = pass_flag
    #for now, ignoring the amendments, but it seems like it could do better with the amendments
    #that would go here (i.e. an else statement saying doc[0] in combined_docs)
    
                   
print("\nTotal bills:", total)
print("Passed bills:", passed)
print("Bill passed ratio:", passed/total)


Total bills: 9520
Passed bills: 6
Bill passed ratio: 0.0006302521008403362


In [3]:
#actually, lets just stick with the given passages for now
leg_data = pd.read_csv("NHLegOverallData09_20.csv") 

#with open('NHStateTargetList09_20.list','rb') as f:
#    event_target = pickle.load(f)

with open('NHWinList09_20.list','rb') as f:
    win_state = pickle.load(f)
    
    
leg_targ = pd.DataFrame({"every stage passed":win_state}) 
#leg_targ = #append bill_id

bill_order = []
bill_target = []
bill_texts = []

leg_targ_old = leg_target
leg_target = {}

for i, bill_id in enumerate(leg_data["bill_id"]):
    if str(bill_id) not in bill_order:
        bill_order.append(str(bill_id))
        bill_target.append(leg_targ["every stage passed"][i])
        leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
        if bill_id in combined_docs:
            bill_texts.append(combined_docs[bill_id])
        else:
            bill_texts.append("")

#for i, bill in enumerate()
print(len(bill_texts))    
print(len(bill_target))
print(len(leg_data))
# 
# for bill_id in bill_order:
#     if bill_id in combined_docs:
#         bill_texts.append(combined_docs[bill_id])
#     else:
#         bill_texts.append("")
# passed = 0
# total = 0
# for i in leg_target:
#     passed+=leg_target[i]
#     total += 1
# print(passed/total)
#print(leg_target)

10612
10612
124345


In [4]:
#print(leg_targ[2100:2120])

In [4]:
#features are in a dictionary. 
#with open('BillFeatures.dict','rb') as f:
#    features = pickle.load(f)
features = leg_data
print(len(leg_targ))
X = pd.DataFrame(features)
X['leg_targ'] = leg_targ
# X_all = X.copy()
# X_all = X_all.set_index('bill_id')


X = X.drop_duplicates(subset=['title']) #should probably be bill_id
X = X.set_index('bill_id')
X = X.drop(columns=['time_t', 'event'])


print(len(X))
#print(X['committee introduced'].head())


bill_order = []
bill_target = []
bill_texts = []

count = 0
for bill_id in X.index:
    if bill_id in combined_docs:
        bill_texts.append(combined_docs[bill_id])
    else:
        bill_texts.append("")
        
print(len(bill_texts))
X['texts'] = bill_texts
# leg_targ_old = leg_target
# leg_target = {}
# for i, bill_id in enumerate(leg_data["bill_id"]):
#     if str(bill_id) not in bill_order:
#         bill_order.append(str(bill_id))
#         bill_target.append(leg_targ["every stage passed"][i])
#         leg_target[str(bill_id)] = leg_targ["every stage passed"][i]
        
#         if bill_id in combined_docs:
#             bill_texts.append(combined_docs[bill_id])
#         else:
#             bill_texts.append("")

X.head()

124345
8908
8908


Unnamed: 0  numeric_date  sponsor_vec  session  year chamber  \
bill_id                                                                 
672808            0   2015.018817     1.000000     1141  2015       H   
683319            7   2015.018817     0.000000     1141  2015       H   
691971           13   2015.018817     0.000000     1141  2015       H   
691453           18   2015.018817     1.000000     1141  2015       H   
673092           24   2015.018817     0.666667     1141  2015       H   

        bill_type                                              title  \
bill_id                                                                
672808         CA  Relating to taxation. Providing that a 3/5 vot...   
683319         CA  Relating to term of office. Providing that beg...   
691971         CA  Relating to voting at elections. Providing tha...   
691453         CA  Relating to qualifications for public office. ...   
673092         CA  Relating to dedicated funds. Providing that fu...   

        proposed_chamber      primary_name  ... primary_party number_sponsors  \
bill_id                                     ...                                 
672808                 H      Jordan Ulery  ...             R               1   
683319                 H      Mario Ratzki  ...             D              10   
691971                 H  Katherine Rogers  ...             D               1   
691453                 H      Jordan Ulery  ...             R               1   
673092                 H      Steven Smith  ...             R               6   

                             committee_introduced month_introduced  \
bill_id                                                              
672808                             Ways and Means                1   
683319   Executive Departments and Administration                1   
691971                               Election Law                1   
691453                               Election Law                1   
673092                             Ways and Means                1   

         primary_leadership  primary_ideology  public_sentiment  press  \
bill_id                                                                  
672808                 0.27               0.5               0.0      0   
683319                 0.27               0.5               0.0      0   
691971                 0.27               0.5               0.0      0   
691453                 0.27               0.5               0.0      0   
673092                 0.27               0.5               0.0      0   

         leg_targ                                              texts  
bill_id                                                               
672808          0  concurrent resolution proposing constitutional...  
683319          0  concurrent resolution proposing constitutional...  
691971          0  concurrent resolution proposing constitutional...  
691453          0  concurrent resolution proposing constitutional...  
673092          0  concurrent resolution proposing constitutional...  

[5 rows x 21 columns]

In [5]:
# #make a mask so we only keep the rows that we actually have text for
# X_mask = [False]*len(X)
# for i, bill_id in enumerate(combined_docs): 
#     try:
#         print(X['bill_id'][X['bill_id'] == bill_id].index)
#         X['bill_id'][X['bill_id'] == bill_id].index[0]
#         X_mask[X['bill_id'][X['bill_id'] == bill_id].index[0]] = True
#     except:
#         print("i:",i,"bill_id:",bill_id)
#         raise IndexError 

# #next step: sift X so that it only contains ones we have docs for
# X_mask = pd.Series(X_mask)
# print(X_mask.iloc[4:9])
# X = X[X_mask]

#X = X.astype({'bill_id': 'int64'}).sort_values('bill_id', ascending=True, ignore_index=True)
#temporarily affix the target
#X['target'] = leg_targ
#X['text'] = bill_texts
X = X.astype({'year': 'int64'}).sort_values('year', ascending=True)#, ignore_index=True)
#date = #make date introduced something like 2018.50, so that we can sort numerically by month too


#print(X.loc[894687]['title'])
X.head()

Unnamed: 0  numeric_date  sponsor_vec  session  year chamber  \
bill_id                                                                 
804171        13253          -1.0          1.0     1201    -1       H   
383650        97043          -1.0          1.0      546    -1     NaN   
842619        16291          -1.0          0.5     1201    -1       H   
842399        16299          -1.0          1.0     1201    -1       H   
132686        73465          -1.0          0.0       54    -1     NaN   

        bill_type                                              title  \
bill_id                                                                
804171          B  Prohibiting a town meeting from materially cha...   
383650          B  Relative to jury trials in class actions broug...   
842619          B  Relative to abuse-deterrent opioid drug products.   
842399          B  Relative to specialty care in drop-in or walk-...   
132686          R  Requesting guidance from the Environmental Pro...   

        proposed_chamber          primary_name  ... primary_party  \
bill_id                                         ...                 
804171                 H        James Spillane  ...             R   
383650                 H         Gene Chandler  ...             R   
842619                 H        Thomas Sherman  ...             D   
842399                 H         Joseph Hannon  ...             R   
132686                 S  Margaret Wood Hassan  ...             D   

        number_sponsors  committee_introduced month_introduced  \
bill_id                                                          
804171               11                  None               -1   
383650                2                  None               -1   
842619                4                  None               -1   
842399                1                  None               -1   
132686                1                  None               -1   

         primary_leadership  primary_ideology  public_sentiment  press  \
bill_id                                                                  
804171                 0.27               0.5               0.0      0   
383650                 0.27               0.5               0.0      0   
842619                 0.27               0.5               0.0      0   
842399                 0.27               0.5               0.0      0   
132686                 0.27               0.5               0.0      0   

         leg_targ                                              texts  
bill_id                                                               
804171          0  act prohibiting town meeting materially changi...  
383650          0  act relative jury trials consumer protection a...  
842619          0  act relative abusedeterrent opioid drug produc...  
842399          0  act relative specialty care dropin walkin clin...  
132686          0                                                     

[5 rows x 21 columns]

In [7]:
len(X)

8908

In [8]:
#Change this so it isn't so memory intensive in the future: combine the text later

# #align the documents with their row to make one complete dataset
# ordered_docs = [""]*len(X)

# #align the targets with their row
# ordered_target = [0]*len(X)
# #print(leg_target)

# #take the bill ids from the dataframe (which are in order)
# ordered_bill_ids = list(X.index.values)
# for i, b_id in enumerate(ordered_bill_ids):
#     try:
#         ordered_docs[i] = combined_docs[b_id]
#     except:
#         ordered_docs[i] = ""
        
#     ordered_target[i] = leg_target[str(b_id)]
#     if str(b_id) == '894761':
#         print(i)
#         print(ordered_target[i])
    

# #make a new column for text    
# X['legtext'] = ordered_docs

In [6]:
#list(X.index.values)[0:10]
#ordered_target[0]
X['month_introduced'] = X['month_introduced'].astype(str)

X.head()

Unnamed: 0  numeric_date  sponsor_vec  session  year chamber  \
bill_id                                                                 
804171        13253          -1.0          1.0     1201    -1       H   
383650        97043          -1.0          1.0      546    -1     NaN   
842619        16291          -1.0          0.5     1201    -1       H   
842399        16299          -1.0          1.0     1201    -1       H   
132686        73465          -1.0          0.0       54    -1     NaN   

        bill_type                                              title  \
bill_id                                                                
804171          B  Prohibiting a town meeting from materially cha...   
383650          B  Relative to jury trials in class actions broug...   
842619          B  Relative to abuse-deterrent opioid drug products.   
842399          B  Relative to specialty care in drop-in or walk-...   
132686          R  Requesting guidance from the Environmental Pro...   

        proposed_chamber          primary_name  ... primary_party  \
bill_id                                         ...                 
804171                 H        James Spillane  ...             R   
383650                 H         Gene Chandler  ...             R   
842619                 H        Thomas Sherman  ...             D   
842399                 H         Joseph Hannon  ...             R   
132686                 S  Margaret Wood Hassan  ...             D   

        number_sponsors  committee_introduced month_introduced  \
bill_id                                                          
804171               11                  None               -1   
383650                2                  None               -1   
842619                4                  None               -1   
842399                1                  None               -1   
132686                1                  None               -1   

        primary_leadership  primary_ideology  public_sentiment  press  \
bill_id                                                                 
804171                0.27               0.5               0.0      0   
383650                0.27               0.5               0.0      0   
842619                0.27               0.5               0.0      0   
842399                0.27               0.5               0.0      0   
132686                0.27               0.5               0.0      0   

         leg_targ                                              texts  
bill_id                                                               
804171          0  act prohibiting town meeting materially changi...  
383650          0  act relative jury trials consumer protection a...  
842619          0  act relative abusedeterrent opioid drug produc...  
842399          0  act relative specialty care dropin walkin clin...  
132686          0                                                     

[5 rows x 21 columns]

In [10]:
# list(combined_docs.keys())[0:10]
# combined_docs[576685]
# #ordered_target
# X['legtext'].head()

In [11]:
#create the entries for the data for demo
# new_test_data = {'bill_id':[],
#                  'sponsor_vec':[],
#                  'chamber':[],
#                  'bill_type':[],
#                  'proposed_chamber':[],
#                  'primary_name':[],
#                  'primary_party':[],
#                  'number_sponsors':[],
#                  'committee_introduced':[],
#                  'month_introduced':[],}            

#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

In [113]:
#select the data that we actually want to train on
#it is probably better to use primary id in the future instead of primary name
#X_new = X[['house/sentate', 'primary name', 'primary party', 'number of co sponsors', 'committee introduced','month introduced']].copy() #, 'committee introduced'
X_new = X[['sponsor_vec','bill_type', 'proposed_chamber',
           'primary_name','primary_party','number_sponsors',
           'committee_introduced','month_introduced', 
           #'primary_leadership','primary_ideology','public_sentiment','press',
          ]].copy()

amount_train = 7100
amount_test = 800

X_dum = pd.get_dummies(X_new)

#can either manually define the features, or uncomment the below feature selection
#X_dum = X_dum[['number of co sponsors', 'house/sentate_S', 'month introduced_01', 'month introduced_05', 'month introduced_06', 'month introduced_12', 'primary party_R']]
# X_dum = X_dum[[
# 'sponsor_vec',
# 'number_sponsors',
# # 'press',
# # 'public_sentiment', #added these three to step around feature selection
# # 'primary_ideology', #
# # 'primary_leadership', #
# 'bill_type_B',
# 'bill_type_CA',
# 'bill_type_CR',
# 'bill_type_R',
# 'proposed_chamber_H',
# 'proposed_chamber_S',
# 'primary_name_Andrew Hosmer',
# 'primary_name_Andrew Manuse',
# 'primary_name_Betsey Patten',
# 'primary_name_Betsi DeVries',
# 'primary_name_Bob Giuda',
# 'primary_name_Bob Odell',
# 'primary_name_Brian Rhodes',
# 'primary_name_Carol McGuire',
# 'primary_name_Charles Weed',
# 'primary_name_Daniel Itse',
# 'primary_name_David Boutin',
# 'primary_name_David Palfrey',
# 'primary_name_David Watters',
# 'primary_name_Dennis Fields',
# 'primary_name_Edward Butler',
# 'primary_name_Frank Sapareto',
# 'primary_name_Gene Chandler',
# 'primary_name_George Lambert',
# 'primary_name_Harold Rice',
# 'primary_name_Ian Raymond',
# 'primary_name_J.R. Hoell',
# 'primary_name_James Headd',
# 'primary_name_James McConnell',
# 'primary_name_James Rausch',
# 'primary_name_Jeb Bradley',
# 'primary_name_Jeff Woodburn',
# 'primary_name_Jeffrey Oligny',
# 'primary_name_Jerry Little',
# 'primary_name_John Barnes',
# 'primary_name_John Burt',
# 'primary_name_John Gallus',
# 'primary_name_John Hunt',
# 'primary_name_John Reagan',
# 'primary_name_Jordan Ulery',
# 'primary_name_Kathleen Hoelzel',
# 'primary_name_Kathleen Sgambati',
# 'primary_name_Keith Ammon',
# 'primary_name_Kenneth Weyler',
# 'primary_name_Lars Christiansen',
# 'primary_name_Laurie Harding',
# 'primary_name_Lou D\'Allesandro',
# 'primary_name_Matthew Houde',
# 'primary_name_Max Abramson',
# 'primary_name_Michael Brewster',
# 'primary_name_Molly Kelly',
# 'primary_name_Naida Kaen',
# 'primary_name_Nancy Stiles',
# 'primary_name_Neal Kurk',
# 'primary_name_None',
# 'primary_name_Paul Mirski',
# 'primary_name_Peggy Gilmour',
# 'primary_name_Regina Birdsell',
# 'primary_name_Richard Marple',
# 'primary_name_Robert Elliott',
# 'primary_name_Robert Hull',
# 'primary_name_Robert Letourneau',
# 'primary_name_Robert Williams',
# 'primary_name_Sharon Carson',
# 'primary_name_Steven Lindsey',
# 'primary_name_Timothy Robertson',
# 'primary_name_Wayne Burton',
# 'primary_party_D',
# 'primary_party_L',
# 'primary_party_None',
# 'primary_party_R',
# 'committee_introduced_Commerce',
# 'committee_introduced_Criminal Justice and Public Safety',
# 'committee_introduced_Energy',
# 'committee_introduced_Energy and Natural Resources',
# 'committee_introduced_Executive Departments and Administration',
# 'committee_introduced_Fish and Game and Marine Resources',
# 'committee_introduced_Health',
# 'committee_introduced_Health and Human Services',
# 'committee_introduced_Judiciary Committee',
# 'committee_introduced_Legislative Administration',
# 'committee_introduced_None',
# 'committee_introduced_Public and Municipal Affairs',
# 'committee_introduced_Resources',
# 'committee_introduced_State-Federal Relations and Veterans Affairs',
# 'committee_introduced_Transportation',
# 'committee_introduced_Transportation and Interstate Cooperation Committee',
# 'month_introduced_1',
# 'month_introduced_11',
# 'month_introduced_12',
# 'month_introduced_2',
# 'month_introduced_3',
# 'month_introduced_4',
# 'month_introduced_5',
# 'month_introduced_6',
# 'month_introduced_7',
# 'month_introduced_8',
# ]] 

#X_dum = pd.concat([X_dum_2, pd.get_dummies(X['event'])], axis=1)
#X_dum = pd.concat([X_dum_2, pd.get_dummies(X['event'])], axis=1)

#print(X_dum.iloc[0])

#ordered_target = pd.get_dummies(X['target'])

X_train_p = X_dum.iloc[0:amount_train]#.iloc
y_train = X['leg_targ'].iloc[0:amount_train]#pd.get_dummies()
X_test_p = X_dum.iloc[amount_train:amount_train+amount_test]#.iloc
y_test = X['leg_targ'].iloc[amount_train:amount_train+amount_test]#pd.get_dummies()




k_select = SelectKBest(f_classif, k=100)
X_train = k_select.fit_transform(X_train_p, y_train)
X_test = k_select.transform(X_test_p)

indices = k_select.get_support(indices=True)
print(X_train_p.iloc[0,indices])
# #indices = 

print(sum(y_train)/len(y_train))
print(sum(y_test)/len(y_test))
#print(X_train[0:10])
#print(X_test[0:10])
#X_train.head()


sponsor_vec            1.0
number_sponsors       11.0
bill_type_B            1.0
bill_type_CA           0.0
bill_type_CR           0.0
                      ... 
month_introduced_4     0.0
month_introduced_5     0.0
month_introduced_6     0.0
month_introduced_7     0.0
month_introduced_8     0.0
Name: 804171, Length: 100, dtype: float64
0.33985915492957747
0.335


/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:114: UserWarning: Features [ 10  16  18  23  32  36  40  71  76  87  88  90  91  92 100 101 102 110
 118 119 134 144 146 152 154 161 165 168 187 198 203 218 225 232 233 235
 237 243 245 252 253 255 264 265 275 276 279 283 299 307 313 319 325 331
 334 336 353 367 384 386 387 390 404 406 416 417 421 434 454 463 469 485
 489 492 502 506 526 531 536 543 544 547 553 566 569 571 577 585 587 589
 600 612 631 637 639 640 642 674 677 688 694 704 709 716 722 725 726 727
 728 731 738 743 747 757 778 782] are constant.
  UserWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:115: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [58]:
for i in indices:
    print("\'"+str(X_train_p.columns[i])+"\',")# X_train_p.iloc[0,i], "\t", 

'sponsor_vec',
'number_sponsors',
'bill_type_B',
'bill_type_CA',
'bill_type_CR',
'bill_type_R',
'proposed_chamber_H',
'proposed_chamber_S',
'primary_name_Andrew Hosmer',
'primary_name_Andrew Manuse',
'primary_name_Betsey Patten',
'primary_name_Betsi DeVries',
'primary_name_Bob Giuda',
'primary_name_Bob Odell',
'primary_name_Brian Rhodes',
'primary_name_Carol McGuire',
'primary_name_Charles Weed',
'primary_name_Daniel Itse',
'primary_name_David Boutin',
'primary_name_David Palfrey',
'primary_name_David Watters',
'primary_name_Dennis Fields',
'primary_name_Edward Butler',
'primary_name_Frank Sapareto',
'primary_name_Gene Chandler',
'primary_name_George Lambert',
'primary_name_Harold Rice',
'primary_name_Ian Raymond',
'primary_name_J.R. Hoell',
'primary_name_James Headd',
'primary_name_James McConnell',
'primary_name_James Rausch',
'primary_name_Jeb Bradley',
'primary_name_Jeff Woodburn',
'primary_name_Jeffrey Oligny',
'primary_name_Jerry Little',
'primary_name_John Barnes',
'primary_name

In [57]:
#X_new.loc[243547]

In [114]:
compressed_legtext = pd.DataFrame(X['texts'].copy())
# print(len(compressed_legtext))
# compressed_legtext['target'] = ordered_target
# compressed_legtext = compressed_legtext.loc[~compressed_legtext.index.duplicated(keep='first')]
# #compressed_legtext.drop_duplicates(subset=compressed_legtext.index, keep='first')
# #compressed_legtext.drop_duplicates(keep='first') 
# print(len(compressed_legtext))
# compressed_legtext.head()

In [16]:
#list(compressed_legtext['legtext'][0:8000])[0:2]


In [115]:
#prepare the documents by using tfidf vectors:

#from https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a
stemmer = SnowballStemmer("english", ignore_stopwords=True)
#adjust this stemmer so that it does more than one word in the tokens

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])

#mini_train = int(amount_train/len(X))*len(compressed_legtext)
#mini_test = int(amount_test/len(X))*len(compressed_legtext)

stemmed_count_vect = StemmedCountVectorizer(stop_words='english', ngram_range=(1,4))
stem_counts = stemmed_count_vect.fit_transform(compressed_legtext['texts'][0:amount_train])
stem_counts_test = stemmed_count_vect.transform(compressed_legtext['texts'][amount_train:amount_train+amount_test])

tfidf = TfidfTransformer()
tfidf_vectors = tfidf.fit_transform(stem_counts)
tfidf_vectors_test = tfidf.transform(stem_counts_test)

In [116]:
#do the final pick after, since we only want to run the above cell once
k_best_tfidf = SelectKBest(f_classif, k=1000)
final_tfidf_features = k_best_tfidf.fit_transform(tfidf_vectors, y_train)
final_tfidf_features_test = k_best_tfidf.transform(tfidf_vectors_test)
word_indices = k_best_tfidf.get_support(indices=True)
#print(word_indices)

In [117]:
#### print(stemmed_count_vect.__dict__.keys())
print(stemmed_count_vect.__dict__['vocabulary_']['rsa'])


2458968


In [20]:
#try withouth the words
#try with the state to state
#train the earlier fit with the overall classifier, it doesnt work with state to state

In [118]:
#combine tfidf legtext and the other features:
legtext_train = final_tfidf_features.todense()
#legtext_train = pd.DataFrame(legtext_train)#*10

legtext_test = final_tfidf_features_test.todense()
#legtext_test = pd.DataFrame(legtext_test)


X_train = np.concatenate([np.array(X_train),legtext_train], axis=1)
X_test = np.concatenate([np.array(X_test),legtext_test], axis=1)
#X_train.head()
#X_train[0]

In [63]:
# def perf_metric(y_test, y_pred):
# #Accuracy
#     correct=0
#     total=0
#     acc=0
#     for i in range(len(y_test)):
#         for j in range(len(y_test.iloc[0])):
#             answer = y_test.iloc[i,j] 
#             real = y_pred[i][j]:
#             if answer == real:
#                 correct+=real 
#         total+=1
#     acc=correct/total*100
#     #print(x,y,len(y_pred),len(y_test))
#     #print('Accuracy:', acc)
#     return acc

# def perf_metric(y_test, y_pred):
    
#     new_mean = np.mean(predicted == y_test)
    
    
#     return acc
import warnings
from sklearn.metrics import classification_report
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)

In [23]:
#print(np.array(y_test.iloc[-8]))
#print(predicted[1001])
#MLPClassifier(random_state=102, max_iter=12) -> 96.75% acc

In [123]:
#train the neural network a bunch
means = []
for i in range(10):
    #print('Training...')
    clf = MLPClassifier(hidden_layer_sizes=(50,),
                        activation='relu',
                        random_state=i+30, max_iter=50,
                        solver='lbfgs', alpha=0.01,
                        tol=0.001, n_iter_no_change=4) #
    #clf = MultinomialNB(alpha=0)#alpha=0.001,DecisionTreeClassifier()#MultinomialNB()##19 ##
    
    
    clf.fit(X_train, y_train)
    #print('Predicting...')
    predicted = clf.predict(X_test)
    predicted_prob = clf.predict_proba(X_test)
    #print(predicted_prob[0])
    #train_prediction = clf.predict(X_train)
    #print('Mean Calculation...')
    #new_mean = perf_metric(y_test, predicted)
    new_mean = np.mean(predicted == y_test)
    means.append(new_mean)
    print(new_mean)
    #print(np.mean(train_prediction == y_train))
    
mean=np.mean(np.array(means))
print("overall mean:", mean)
print(classification_report(y_test, predicted, digits=4))


0.96875
0.9675
0.9675
0.9675
0.96625
0.96875
0.96875
0.9675
0.9675
0.9675
overall mean: 0.9677499999999999
              precision    recall  f1-score   support

           0     0.9534    1.0000    0.9761       532
           1     1.0000    0.9030    0.9490       268

    accuracy                         0.9675       800
   macro avg     0.9767    0.9515    0.9626       800
weighted avg     0.9690    0.9675    0.9671       800



In [25]:
print(np.mean(y_train))
np.mean(y_test)

0.33985915492957747


0.335

In [26]:
#actualMNBclf = MNBclf
MNBclf = clf
print(type(clf))
#find bill_id from mongoDB:
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

bill_id = 1287298
HB_test = X_dum.loc[[bill_id]]#.reshape(1, -1)
print("All data on bill_id", bill_id)
print(X.loc[[bill_id]])
HB_test = k_select.transform(HB_test)
indices = k_select.get_support(indices=True)
print("\n\nUsed data on bill_id", bill_id)
print(X_dum.loc[[bill_id]].iloc[0,indices])

HB_test = np.array(HB_test)

HB_legtext_counts = stemmed_count_vect.transform(X[['texts']].loc[bill_id])
HB_legtext_tfidf = tfidf.transform(HB_legtext_counts)
HB_legtext = k_best_tfidf.transform(HB_legtext_tfidf).todense()
HB_legtext = np.array(HB_legtext)

HB_data = np.concatenate([HB_test, HB_legtext], axis=1)
#print(HB_data)

HB1389_prob = MNBclf.predict_proba(HB_data)
#HB1680_prob = MNBclf.predict_proba(X_dum.loc[1294143])

print(HB1389_prob)
#show all the data
print(X.loc[bill_id])

<class 'sklearn.neural_network._multilayer_perceptron.MLPClassifier'>
All data on bill_id 1287298
         Unnamed: 0  numeric_date  sponsor_vec  session  year chamber  \
bill_id                                                                 
1287298      119847   2019.919355          0.0     1729  2019       H   

        bill_type                                              title  \
bill_id                                                                
1287298         B  Relative to criminal penalties for not providi...   

        proposed_chamber      primary_name primary_id primary_party  \
bill_id                                                               
1287298                H  Katherine Rogers      14480             D   

         number_sponsors committee_introduced month_introduced  leg_targ  \
bill_id                                                                    
1287298                3                 None               12         0   

                      

In [113]:
def get_actions(action_clf, data, desired_probability, epochs=10, alpha=0.1, epsilon=0.0001,
                first_k = 100, labels=None, verbose=False): 
    #In the future, include a feature_mask instead of first_k
    #This returns all actions. May want to create something that does top-n actions.
    """
    action_clf: a classifier that is compatible with sklearn, typically the overall classifier
    data: a single row of bill data (numpy array)
    desired_probability: an float between 0 and 1 inclusive
    first_k: only act on (and print) the first k actions
    labels: the named labels of the inputs
    base_labels: stemmed labels (i.e. not split into pd dummies)
    epochs: number of epochs to run iterative gradient descent
    alpha: step size to move in the direction of the gradient
    epsilon: the size of the perturbation of the inputs
    verbose: if True, will print predicted probability at each epoch, and print final actions
    """
    
    initial_prob = action_clf.predict_proba(data)
    
    if verbose:
        print("Initial probability of passing:", initial_prob)
    
    DZDY = desired_probability - initial_prob #gradient of the loss wrt the output. Scalar for a single input
    DZDX = np.zeros_like(data) #empty array of the same size as data (i.e. x)
    DZDX_c = np.zeros_like(data)
    
    x_p = data.copy()
    x_n = data.copy()

    for epoch in range(epochs):
        #can handle batches of data (multiple bills):
        for t, row in enumerate(x_p): 
            #since our dataset is metadata followed by tfidf words, and we can't really affect the words yet:
            for i in range(first_k): 
                #perturb the inputs a little bit
                x_p[t][i] += epsilon
                x_n[t][i] -= epsilon
                #predict from the perturbed input
                f_p = action_clf.predict_proba(x_p)
                f_n = action_clf.predict_proba(x_n)
                #this is basically newtons method to find the gradient
                DZDX[t][i] = np.sum(DZDY*((f_p-f_n)/(2*epsilon)))
                # reset the tensors to their original state 
                x_p[t][i] -= epsilon
                x_n[t][i] += epsilon
                
        DZDX_c += DZDX
        adjusted_data = data + DZDX_c
        adjusted_data[adjusted_data < 0] = 0
        DZDX_c = adjusted_data - data
        
        new_prob = action_clf.predict_proba(data + DZDX_c)
        
        if verbose:
            print("Prob on epoch", epoch, ":", new_prob)
            
        DZDY = desired_probability - new_prob
    
    if verbose:
        print("Final actions:")
        for i in range(first_k):
            print("{0:.5f}".format(DZDX_c[0][i]), "\t", labels[i])

    return DZDX_c
    
    
def get_recommendations(clf, data, desired_probability, DZDX_c, label_action_count, 
                        metadata_num, labels, base_labels, verbose=False):
    """
    get_recommendations basically turns get_actions into useful information
    clf: sklearn classifier, typically the overall classifier, built for a MLP (neural network)
    data: 
    desired_probability: between 0 and 1
    DZDX_c: the cumulative change in the variable vector as produced by get_actions
    metadata_num: number of selected dummy metadata features (selects the first metadata_num features)
    base_labels: the base labels that pd.dummies has made expanded into one-hot encodings
    label_action_count: an iterable of ints indicating the number of options to give for that feature. 
        Has the same length as base_labels. 
        ex: base_labels = [committee, primary_sponsor], label_action_count = [3, 10]
    """
    feature_importances = {base_label:[] for base_label in base_labels}
    for i, base_label in enumerate(base_labels):
        #this works because pd dummies puts things in order
        for j in range(metadata_num):
            if labels[j][0:len(base_label)] == base_label:
                #store in feature_importances such that they can be sorted later
                feature_importances[base_label].append((labels[j], DZDX_c[0][j]))
            
    #sort each label by absolute feature importance
    for i, label in enumerate(feature_importances):
        #sort by the absolute value of that feature importance 
        feature_importances[label].sort(key=lambda x: abs(x[1]), reverse=True)
    
    final_actions = {base_label:[] for base_label in base_labels}
    for i, count in enumerate(label_action_count):
        final_actions[base_labels[i]] = feature_importances[base_labels[i]][0:count]
    
    if verbose:
        for key in final_actions.keys():
            
            if final_actions[key] != []:
                print(key)
                for i in final_actions[key]:
                    print("\t", "{0:.5f}".format(i[1]), "\t\t", i[0])

    return final_actions
    
        
#to do:
#test the above functions
#
def what_if_from_recs(clf, data, modifications, top_n=10):
    """
    A function that takes a classifier, some data, and modifications to that data, and 
    Produces a cartesian product of modifications. i.e. if modifications is {'a':['b','c'],'d':['e','f','g']},
    then 6 probabilities will be produced
    
    send in modicications produced by get_recommendations
    """
    modification_calls = modifications.keys()
    
    for call in modification_calls:
        modifications[call].append('') #the 'no change' call
    
    
    
    return  



def what_if_from_recs_helper(clf, data, modifications, top_n=10, score_dict=):
    pass
        
        

def individual_what_if_from_recs(clf, data, modifications, top_n=10):
    """
    A function that takes a classifier, some data, and modifications to that data, and 
    Produces a cartesian product of modifications. i.e. if modifications is {'a':['b','c'],'d':['e','f','g']},
    then 6 probabilities will be produced
    
    send in modicications produced by get_recommendations
    """
    modification_calls = modifications.keys()
    
    for call in modification_calls:
        for modification in modifications[call]:
            new_data = data[modification[0]]
            probability = clf.predict_proba()
            
           
    
    
    return   


            
    

        
        
        
def what_if(clf, data, modifications):
    """
    A function that takes a classifier, some data, and modifications to that data, and 
    Produces a cartesian product of modifications. i.e. if modifications is {'a':['b','c'],'d':['e','f','g']},
    then 6 probabilities will be produced
    """
    return
    
    
    

In [66]:
X_new.columns

Index(['sponsor_vec', 'chamber', 'bill_type', 'proposed_chamber',
       'primary_name', 'primary_party', 'number_sponsors',
       'committee_introduced', 'month_introduced'],
      dtype='object')

In [1]:
#save the overall clf
#overall_clf = clf #this has been run and saved

#HB1680: bill_id is 1294143
bill_id = 1287298
HB_test = X_dum.loc[[bill_id]]#.reshape(1, -1)
# print("All data on bill_id", bill_id)
# print(X.loc[[bill_id]])
HB_test = k_select.transform(HB_test)
indices = k_select.get_support(indices=True)
#print("\n\nUsed data on bill_id", bill_id)
#print(X_dum.loc[[bill_id]].iloc[0,indices])

HB_test = np.array(HB_test)

HB_legtext_counts = stemmed_count_vect.transform(X[['texts']].loc[bill_id])
HB_legtext_tfidf = tfidf.transform(HB_legtext_counts)
HB_legtext = k_best_tfidf.transform(HB_legtext_tfidf).todense()
HB_legtext = np.array(HB_legtext)
HB_data = np.concatenate([HB_test, HB_legtext], axis=1)
#print(HB_data)

HB_prob = overall_clf.predict_proba(HB_data)

action_vector = get_actions(overall_clf, HB_data, 1, first_k = 100, labels=X_dum.columns[indices],  
                            epochs=100, alpha=0.1, epsilon=0.0001, verbose=True) #in the future, include a feature_mask instead of hard-coding features

# Order: 'sponsor_vec', 'chamber', 'bill_type', 'proposed_chamber', 'primary_name', 
# 'primary_party', 'number_sponsors','committee_introduced', 'month_introduced'

label_action_count = [1,0,0,1,5,2,1,0,3] #probably change this to a dictionary
print("Suggested actions:\n")
recs_from_action_count = get_recommendations(overall_clf, HB_data, 1, action_vector, label_action_count,
                                             100, X_dum.columns[indices], X_new.columns, verbose=True)
#base_labels=X.columns,


NameError: name 'X_dum' is not defined

In [28]:
#y_test
def vec_to_state(prediction_vectors, example_output_df):
    index = np.array(example_output_df.columns)
    print('index[0:10]',index[0:10])
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    prediction_vectors = prediction_vectors == 1
    predictions = []
    for prediction in prediction_vectors:
        #print(prediction)
        
        if len(index[prediction]) == 0:
            append_value = [0]
        else:
            append_value = index[prediction][0]
            
        print('append_value:', append_value)
        predictions.append(append_value)
        
    return predictions

def prob_vec_to_n_states(prediction_vector, example_output_df, n=1):
    #return the names, and probabilities in order
    index = np.array(example_output_df.columns)
    
    #print(index)
    #prediction_vectors = np.array([[1,0,0,0,0], [0,0,1,0,0], [0,0,0,0,1],[0,1,0,0,0]]) == 1
    #print(prediction_vectors)
    ind = np.argpartition(prediction_vector, -n)[-n:]
    sorted_ind = ind[np.argsort(prediction_vector[ind])][::-1]
    
    predictions = []
    for index_s in sorted_ind:
        append_value = index[index_s]
            
        print('append_value:', append_value)
        predictions.append((append_value, prediction_vector[index_s]))
        
    return predictions
    


#pd.get_dummies(X['event'])
def state_to_vec(current_state, example_output_df):
    X_index = np.array(example_output_df.columns)
    indices = current_state #a string

    #for index in indices:
    location = X_index == index
    location = location.astype(int)
        #print(location)
    return(location)
    

In [50]:
#pd.get_dummies(X[['event', 'month_introduced']]).columns

In [ ]:
#HB1680: bill_id is 1294143
#HB1389: bill_id is 1287298

#save the state to state data so we can make a prediction on it next
#sta
#see what is actually in there
#print(X_dum.loc[1294143])
#print(X_dum.loc[1287298])
print(X.loc[1294143])
#prediction_vector = StSclf.predict_proba(X_dum.loc[1294143])[-1]#1294143])
#make an initial prediction of the next step
#print(prob_vec_to_n_states(prediction_vector, y_test, n=5))

#predictions = vec_to_state(prediction_vectors, y_test)
#print(predictions)
#X_dum.head()

In [ ]:
#DTClf has been saved
#DTClf = clf

In [ ]:


#print(indices)
#print(word_indices)
type(clf.feature_log_prob_)

In [ ]:

num_k = 30
num_words = 1000

#only run this if using MultinomialNB classifier
feature_indices = indices
pass_probs = 10**clf.feature_log_prob_[1] - 10**clf.feature_log_prob_[0]

metadata_feature_pass_probs = {X_dum.columns[feature]:pass_probs[i] for i, feature in enumerate(feature_indices)}
inverted_vocab = stemmed_count_vect.__dict__['vocabulary_']
vocab = {inverted_vocab[key]:key for key in inverted_vocab.keys()}

text_feature_pass_probs = {vocab[feature]:pass_probs[i+num_k] for i, feature in enumerate(word_indices)}
#stemmed_count_vect.__dict__['vocabulary_']['rsa']

all_features = {**metadata_feature_pass_probs, **text_feature_pass_probs}

list_features = list(all_features.keys())

sorted_features = sorted(list_features, key=lambda x: all_features[x], reverse=True)
bad_features = sorted(list_features, key=lambda x: all_features[x], reverse=False)
#print(sorted_features)

#vocab_dict = {indices[i]:X_dum.keys()[indices][i] for i, _ in enumerate(indices)}
#print(vocab_dict)

print("Increase the amount of these features:")
for i, feature in enumerate(sorted_features):
    if i >= 50:
        break
    print("{0:.10f}".format(all_features[feature]), "\t", feature)

print("\n\nDecrease the amount of these features:")
for i, feature in enumerate(bad_features):
    if i >= 50:
        break
    print("{0:.10f}".format(all_features[feature]), "\t", feature)

In [ ]:
print(len(predicted))
print()
for i, prediction in enumerate(predicted):
    if i == 50:
        break
    print("real:", y_test[i],"\tpredicted:", predicted_prob[i][1])#, "\tdata:", X_dum.iloc[amount_train:amount_train+amount_test,indices])
